In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

import os

import statsmodels.formula.api as smf
from src.utils.ECG_data_loading import ECG_classification_dataset_with_peak_features
from torch.utils.data import DataLoader

%load_ext autoreload
%autoreload 2

C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: [WinError 127] The specified procedure could not be found
  warn(f"Failed to load image Python extension: {e}")


functorch not installed.


# Load ECG Data

In [2]:
data_folder = ""
data_chunk_folder = "ecg-pat40-tch-sinus_jet_lead2"
load_training_data = False
if load_training_data:
    data_chunk_list = []
    for data_filename in os.listdir(os.path.join(data_folder, data_chunk_folder)):
        data_chunk_list.append(pd.read_csv(os.path.join(data_folder, data_chunk_folder, data_filename)))
    feature_df_all_selected_with_ecg = pd.concat(data_chunk_list, axis=0)
    channel_ID = 2
    feature_with_ecg_df_train_single_lead = feature_df_all_selected_with_ecg.query(f"patient_ID in {patient_ID_list_train}").query(f"channel_ID == {channel_ID}")
    
    train_dataset = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_train_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300)
    train_loader = DataLoader(
        train_dataset, batch_size=128, num_workers=1, shuffle=False, drop_last=False
    )

In [3]:
feature_with_ecg_df_val_single_lead = pd.read_csv("feature_with_ecg_df_val_lead2.csv")
feature_with_ecg_df_test_single_lead = pd.read_csv("feature_with_ecg_df_test_lead2.csv")
feature_with_ecg_df_test_single_lead

,cycle_ID,patient_ID,interval_ID,block_ID,channel_ID,segment_ID,start_ID,end_ID,r_ID,r_t,...,ecg295,ecg296,ecg297,ecg298,ecg299,ecg300,r_ID_abs,r_ID_abs_ref,selected,duration
0,1,771495,8748755,1,2,1,1,66705,820,2015-12-22 23:12:04.099549952,...,1.290715,1.403295,1.606575,1.857944,2.039397,2.014317,820,820,True,277.528
1,2,771495,8748755,1,2,1,1,66705,907,2015-12-22 23:12:04.461459968,...,1.152530,1.248552,1.434638,1.692337,1.924521,1.995100,907,907,True,277.528
2,3,771495,8748755,1,2,1,1,66705,994,2015-12-22 23:12:04.823380224,...,0.954466,1.053145,1.247815,1.520770,1.775172,1.874207,994,994,True,277.528
3,4,771495,8748755,1,2,1,1,66705,1085,2015-12-22 23:12:05.201929984,...,1.468770,1.449762,1.533679,1.723174,1.924213,1.983855,1085,1085,True,277.528
4,5,771495,8748755,1,2,1,1,66705,1167,2015-12-22 23:12:05.543040000,...,1.150959,1.294285,1.518599,1.771597,1.948649,1.938984,1167,1167,True,277.528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114407,308,536886,8748726,1,2,2,4442,31910,27061,2015-01-06 01:31:59.848000000,...,0.521465,0.680327,0.817151,0.849854,0.728921,0.478241,31502,31502,True,132.808
114408,309,536886,8748726,1,2,2,4442,31910,27157,2015-01-06 01:32:00.247380224,...,0.482723,0.645244,0.786819,0.825585,0.711836,0.468551,31598,31598,True,132.808
114409,310,536886,8748726,1,2,2,4442,31910,27253,2015-01-06 01:32:00.646749952,...,0.497711,0.666044,0.814049,0.855897,0.739084,0.487000,31694,31694,True,132.808
114410,311,536886,8748726,1,2,2,4442,31910,27349,2015-01-06 01:32:01.046129920,...,0.541680,0.717542,0.873392,0.916993,0.791512,0.521507,31790,31790,True,132.808


In [4]:
val_dataset = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_val_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300)

test_dataset = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_test_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300)

val_loader = DataLoader(
    val_dataset, batch_size=128, num_workers=1, shuffle=False, drop_last=False
)
test_loader = DataLoader(
    test_dataset, batch_size=128, num_workers=1, shuffle=False, drop_last=False
)

## Save ECG data as dict for lookup later

In [5]:
from torch.utils.data import Dataset
import torch
from src.utils.ECG_data_loading import ECG_classification_dataset_with_peak_features, Transform_frame, Normalize

class ECG_classification_dataset_with_peak_features(Dataset):
    def __init__(self, feature_df_all_selected_p_ind_with_ecg, ecg_resampling_length_target=300,
                 peak_loc_name="p_ind_resampled", label_name="label", short_identifier_list=None,
                 peak_feature_name_list=None, shift_signal=False, shift_amount=None, normalize_signal=False,
                 transforms=None, dataset_name="ecg-TCH-40_patient-20220201", aug_prob=0,
                 return_original_signal=None, return_id_vec=False):
        """
            normalize_signal: Normalize each individual signal to 0 - 1 range
        """
        # print(f"ecg_resampling_length_target: {ecg_resampling_length_target}")
        if short_identifier_list is None:
            short_identifier_list = ['patient_ID', 'interval_ID', 'block_ID', 'channel_ID', 'r_ID_abs', 'label',
                                     'r_ID_abs_ref']
        if peak_feature_name_list is None:
            peak_feature_name_list = ["p_prom_med", "pr_int_iqr"]

        if transforms is None:
            self.transforms = []
        else:
            if isinstance(transforms, str):
                transforms = [transforms]
            self.transforms = [Lower(ele) for ele in transforms]

        self.dataset_name = dataset_name
        self.short_identifier_list = short_identifier_list
        self.peak_feature_name_list = peak_feature_name_list
        self.label_name = label_name
        self.device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
        self.feature_df_all_selected_p_ind_with_ecg = feature_df_all_selected_p_ind_with_ecg
        self.ecg_resampling_length = 300
        self.ecg_resampling_length_target = ecg_resampling_length_target
        self.ecg_colnames = [f"ecg{i + 1}" for i in range(self.ecg_resampling_length)]
        self.peak_loc_name = peak_loc_name
        self.ecg_mat = self.feature_df_all_selected_p_ind_with_ecg[self.ecg_colnames].values
        self.peak_label_list = self.feature_df_all_selected_p_ind_with_ecg[self.peak_loc_name].values
        self.label_list = self.feature_df_all_selected_p_ind_with_ecg[self.label_name].values
        self.short_identifier_mat = self.feature_df_all_selected_p_ind_with_ecg[self.short_identifier_list].values
        self.peak_feature_mat = self.feature_df_all_selected_p_ind_with_ecg[self.peak_feature_name_list].values
        self.return_id_vec = return_id_vec

        self.shift_signal = shift_signal
        self.shift_amount = shift_amount
        self.normalize_signal = normalize_signal
        self.aug_prob = aug_prob
        if self.shift_signal:
            if self.shift_amount is None:
                self.shift_amount = 0
            self.ecg_mat -= self.shift_amount  # Shift ECG to 0 baseline

        if self.normalize_signal:
            ecg_min = np.min(self.ecg_mat, axis=1)[:, np.newaxis]
            ecg_max = np.max(self.ecg_mat, axis=1)[:, np.newaxis]
            self.ecg_mat = (self.ecg_mat - ecg_min) / (ecg_max - ecg_min)

        if return_original_signal is None:
            self.return_original_signal = True
            if len(self.transforms) == 0 or \
                    (len(self.transforms) == 1 \
                     and Lower(self.transforms[0]) in [
                         Lower("Identity"), Lower("SelectedAug_20221025"), Lower("SelectedAug_20221029")]):
                self.return_original_signal = False
        else:
            self.return_original_signal = return_original_signal

    def obtain_perturbed_frame(self, frame):
        frame = Transform_frame(frame, transforms=self.transforms, dataset_name=self.dataset_name,
                                aug_prob=self.aug_prob)
        return frame

    def __len__(self):
        return len(self.feature_df_all_selected_p_ind_with_ecg)

    def __getitem__(self, idx):
        X = self.ecg_mat[idx, :]
        if self.ecg_resampling_length_target != self.ecg_resampling_length:
            X = resample_poly(X, self.ecg_resampling_length_target, self.ecg_resampling_length,
                              padtype="line")
        X = Normalize(X)
        X_aug = self.obtain_perturbed_frame(X)
        peak_idx = self.peak_label_list[idx]
        label = self.label_list[idx]
        id_vec = self.short_identifier_mat[idx, :]
        peak_features = self.peak_feature_mat[idx, :]

        # return X[np.newaxis, :], peak_idx, label, id_vec, peak_features[np.newaxis, :]
        if self.return_id_vec:
            return X_aug[np.newaxis, :], id_vec, label
        else:
            if self.return_original_signal:
                return (X[np.newaxis, :], X_aug[np.newaxis, :]), label
            else:
                return X_aug[np.newaxis, :], label

In [ ]:
train_dataset_with_id = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_train_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300,
                                                                     return_id_vec=True)

In [6]:
val_dataset_with_id = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_val_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300,
                                                                     return_id_vec=True)

In [7]:
test_dataset_with_id = ECG_classification_dataset_with_peak_features(feature_with_ecg_df_test_single_lead,
                                                                     shift_signal=False,
                                                                     shift_amount=0,
                                                                     normalize_signal=True,
                                                                     ecg_resampling_length_target=300,
                                                                     return_id_vec=True)

In [8]:
def Get_dataset_id_dict(dataset):
    dataset_id_dict = {}
    for i, (x, id_vec, y) in enumerate(dataset):
        ECG_signal = x.ravel()
        ECG_signal_str = "_".join([f"{ele:.3f}" for ele in ECG_signal])
        id_dict = dict(zip(dataset.short_identifier_list, id_vec))
        id_dict["label"] = y
        dataset_id_dict[ECG_signal_str] = id_dict
    return dataset_id_dict

# dataset_id_dict_train = Get_dataset_id_dict(train_dataset_with_id)
dataset_id_dict_val = Get_dataset_id_dict(val_dataset_with_id)
dataset_id_dict_test = Get_dataset_id_dict(test_dataset_with_id)

In [9]:
def Lookup_ECG(ECG_signal, dataset_id_dict):
    ECG_signal = ECG_signal.ravel()
    ECG_signal_str = "_".join([f"{ele:.3f}" for ele in ECG_signal])
    id_dict = dataset_id_dict.get(ECG_signal_str)
    return id_dict

# Load model

In [10]:
from torch.utils.data import DataLoader
from src.methods.supervised_1d import SupervisedModel_1D
from src.models.CLOCS_1D import cnn_network_contrastive
import torch.nn as nn
from pathlib import Path
from src.utils.metrics import accuracy_at_k, weighted_mean, AUROC
from src.utils.ECG_data_loading import *
import time

In [23]:
model_dict = {}

In [24]:
model_ID = "cqpywv9j"
model_key = "ClocsCNN_0"

embedding_dim = 256
model = cnn_network_contrastive(zero_init_residual=None, embedding_dim=embedding_dim,
                   stride=2, c4_multiplier=3)
model.fc = nn.Identity()
ckpt_dir = Path(f"trained_models/linear/{model_ID}") # test_auroc = 0.9631
max_val_auroc = max([float(ele.split("=")[-1].replace(".ckpt", "")) \
                     for ele in os.listdir(ckpt_dir) if "=" in ele and "val_auroc" in ele])
ckpt_path = [ckpt_dir / ckpt for ckpt in os.listdir(ckpt_dir) if ckpt.endswith(f"val_auroc={max_val_auroc}.ckpt")][0]
args_path = ckpt_dir / "args.json"
print(f"ckpt_path: {ckpt_path}")
with open(args_path) as f:
    method_args = json.load(f)
model.pretrained_occlusion_model_dict = None
method_args["backbone"] = model
model_loaded = SupervisedModel_1D.load_from_checkpoint(
    ckpt_path, strict=False, **method_args
)
model_dict[model_key] = model_loaded

2 0.1 0.1 0.1 1 256 CLOCS 
Using kernel size: 7
c1, c2, k, s, c3, c4, c4_multiplier, embedding_dim: (1, 4, 7, 2, 16, 32, 3, 256)
ckpt_path: trained_models\linear\cqpywv9j\supervised-clocsCNN_1D-ECG_normalized-20221202_v30-maxep200-early_stopping_patience40-mixup-label_smoothing-aug_selected_20221029_prob0__2-cqpywv9j-epoch=21-val_auroc=0.9505.ckpt


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [25]:
model_ID = "g09je3xq"
model_key = "ClocsCNN_1"

embedding_dim = 256
model = cnn_network_contrastive(zero_init_residual=None, embedding_dim=embedding_dim,
                   stride=2, c4_multiplier=3)
model.fc = nn.Identity()
ckpt_dir = Path(f"trained_models/linear/{model_ID}") # test_auroc = 0.9631
max_val_auroc = max([float(ele.split("=")[-1].replace(".ckpt", "")) \
                     for ele in os.listdir(ckpt_dir) if "=" in ele and "val_auroc" in ele])
ckpt_path = [ckpt_dir / ckpt for ckpt in os.listdir(ckpt_dir) if ckpt.endswith(f"val_auroc={max_val_auroc}.ckpt")][0]
args_path = ckpt_dir / "args.json"
print(f"ckpt_path: {ckpt_path}")
with open(args_path) as f:
    method_args = json.load(f)
model.pretrained_occlusion_model_dict = None
method_args["backbone"] = model
model_loaded = SupervisedModel_1D.load_from_checkpoint(
    ckpt_path, strict=False, **method_args
)
model_dict[model_key] = model_loaded

2 0.1 0.1 0.1 1 256 CLOCS 
Using kernel size: 7
c1, c2, k, s, c3, c4, c4_multiplier, embedding_dim: (1, 4, 7, 2, 16, 32, 3, 256)
ckpt_path: trained_models\linear\g09je3xq\supervised-clocsCNN_1D-ECG_normalized-20221202_v30-maxep200-early_stopping_patience40-mixup-label_smoothing-aug_selected_20221029_prob0__2-g09je3xq-epoch=04-val_auroc=0.9502.ckpt


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [26]:
model_ID = "s6qzuo6i"
model_key = "ClocsCNN_2"

embedding_dim = 256
model = cnn_network_contrastive(zero_init_residual=None, embedding_dim=embedding_dim,
                   stride=2, c4_multiplier=3)
model.fc = nn.Identity()
ckpt_dir = Path(f"trained_models/linear/{model_ID}") # test_auroc = 0.9631
max_val_auroc = max([float(ele.split("=")[-1].replace(".ckpt", "")) \
                     for ele in os.listdir(ckpt_dir) if "=" in ele and "val_auroc" in ele])
ckpt_path = [ckpt_dir / ckpt for ckpt in os.listdir(ckpt_dir) if ckpt.endswith(f"val_auroc={max_val_auroc}.ckpt")][0]
args_path = ckpt_dir / "args.json"
print(f"ckpt_path: {ckpt_path}")
with open(args_path) as f:
    method_args = json.load(f)
model.pretrained_occlusion_model_dict = None
method_args["backbone"] = model
model_loaded = SupervisedModel_1D.load_from_checkpoint(
    ckpt_path, strict=False, **method_args
)
model_dict[model_key] = model_loaded

2 0.1 0.1 0.1 1 256 CLOCS 
Using kernel size: 7
c1, c2, k, s, c3, c4, c4_multiplier, embedding_dim: (1, 4, 7, 2, 16, 32, 3, 256)
ckpt_path: trained_models\linear\s6qzuo6i\supervised-clocsCNN_1D-ECG_normalized-20221202_v30-maxep200-early_stopping_patience40-mixup-label_smoothing-aug_selected_20221029_prob0__2-s6qzuo6i-epoch=162-val_auroc=0.9495.ckpt


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [27]:
model_dict.keys()

dict_keys(['ClocsCNN_0', 'ClocsCNN_1', 'ClocsCNN_2'])

# Get softmax scores from models

In [28]:
from sklearn import metrics
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

In [33]:
def Get_roc_curve_df_from_model_and_data_loader(model, data_loader, target_fpr=0.1, target_threshold=None):
#     data_loader = test_loader

    X_all_list = []
    X_embedding_all_list = []
    y_true_all_list = []
    X_logits_all_list = []
    scores_all_list = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()
    auroc = AUROC(pos_label=1)
    softmax = torch.nn.Softmax(dim=1)

    for i, data in enumerate(data_loader):
#         print(f"Batch {i} / {len(data_loader)}")
        X, y = data
        X, y = X.to(device), y.to(device)
        X_all_list.append(X)
        X_embedding = model.backbone(X)
        X_logits = model.classifier(X_embedding)
    #     print(X_embedding.shape, y.shape)
    #     print(y)
    #     print(X_logits)
        X_embedding_all_list.append(X_embedding)
        y_true_all_list.append(y)
        X_logits_all_list.append(X_logits)

        scores = softmax(X_logits)[:, 1].detach().cpu()
        scores_all_list.append(scores)
        auroc.update(scores, y.detach().cpu())
    #     break

    auroc_value = auroc.compute()
    auroc.reset()
    print(f"auroc_value = {auroc_value:.4f}")
    X_all = torch.cat(X_all_list, dim=0)
    y_true_all = torch.cat(y_true_all_list, dim=0)
    scores_all = torch.cat(scores_all_list, dim=0)
    X_embedding_all = torch.cat(X_embedding_all_list, dim=0)
    X_embedding_all_np = X_embedding_all.detach().numpy()
    y_true_all_np = y_true_all.detach().numpy()
    scores_all_np = scores_all.detach().numpy()
#     X_all.shape, y_true_all.shape, scores_all.shape, X_embedding_all.shape

    y_scores = scores_all
    y_true = y_true_all_np
    
#     auroc = roc_auc_score(y_true, y_scores, average="weighted")
    auroc = auroc_value
    fpr, tpr, thresholds = roc_curve(y_true, y_scores, pos_label=1)
    roc_curve_df = pd.DataFrame([fpr, tpr, thresholds]).T
    roc_curve_df.columns=["fpr", "tpr", "thresholds"]
    if target_threshold is not None:
        closest_threshold_idx = np.argsort(np.abs(roc_curve_df["thresholds"] - target_threshold))[0]
    else:
        closest_threshold_idx = np.argsort(np.abs(roc_curve_df["fpr"] - target_fpr))[0]
    selected_threshold = roc_curve_df["thresholds"][closest_threshold_idx]
    selected_fpr = roc_curve_df["fpr"][closest_threshold_idx]
    roc_curve_results_dict = {"roc_curve_df": roc_curve_df, "selected_threshold": selected_threshold, "selected_fpr": selected_fpr, "auroc": auroc, "y_scores": y_scores, "y_true": y_true}
    return roc_curve_results_dict

## Train an ensemble model using val data

In [120]:
dataset_dict = {
    "dev": None,
    "val": val_dataset,
    "test": test_dataset
}

In [121]:
dataset_id_dict_dict = {
    "dev": None,
    "val": dataset_id_dict_val,
    "test": dataset_id_dict_test
}

In [31]:
data_loader_dict = {
    "dev": None,
    "val": val_loader,
    "test": test_loader
}
roc_curve_results_dict_dict_dict = {"dev": {}, "val": {}, "test": {}}

In [98]:
target_fpr = 0.05
mode = "val"
for model_key in model_dict:
    roc_curve_results_dict_dict_dict[mode][model_key] = Get_roc_curve_df_from_model_and_data_loader(
        model_dict[model_key], data_loader_dict[mode], target_fpr=target_fpr, target_threshold=None)

C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


auroc_value = 0.9505


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


auroc_value = 0.9502


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


auroc_value = 0.9495


In [88]:
target_fpr = 0.05
mode = "test"
for model_key in model_dict:
    roc_curve_results_dict_dict_dict[mode][model_key] = Get_roc_curve_df_from_model_and_data_loader(
        model_dict[model_key], data_loader_dict[mode], target_fpr=target_fpr, target_threshold=None)

C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


auroc_value = 0.9395


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


auroc_value = 0.9519


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


auroc_value = 0.9506


### Simple average

In [106]:
mode = "val"

y_score_df_dict = {}
y_true = roc_curve_results_dict_dict_dict[mode][list(model_dict.keys())[0]]["y_true"]
y_score_total = np.zeros_like(y_true)
counter = 0
for model_key in model_dict:
    y_score_df_dict[f"{model_key}_scores"] = roc_curve_results_dict_dict_dict[mode][model_key]["y_scores"]
    print(type(y_score_df_dict[f"{model_key}_scores"]))
    print(y_score_df_dict[f"{model_key}_scores"].numpy())
    y_score_total = y_score_total + y_score_df_dict[f"{model_key}_scores"].numpy()
    counter += 1
    
y_proba_1 = y_score_total / counter
y_proba_1

<class 'torch.Tensor'>
[0.22374928 0.27148932 0.24138899 ... 0.27909753 0.2880153  0.2674887 ]
<class 'torch.Tensor'>
[0.18680769 0.24083598 0.25396523 ... 0.2737158  0.27627054 0.29468516]
<class 'torch.Tensor'>
[0.2279379  0.3139756  0.3716897  ... 0.2673601  0.27057743 0.2856751 ]


array([0.21283163, 0.27543363, 0.28901464, ..., 0.27339114, 0.27828776,
       0.28261632])

In [107]:
auroc = AUROC(pos_label=1)
auroc.update(torch.from_numpy(y_proba_1), torch.from_numpy(y_true))
auroc_value = auroc.compute()
auroc.reset()
print(f"auroc_value = {auroc_value:.4f}")

auroc_value = 0.9521


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


#### Test

In [108]:
mode = "test"

y_score_df_dict = {}
y_true = roc_curve_results_dict_dict_dict[mode][list(model_dict.keys())[0]]["y_true"]
y_score_total = np.zeros_like(y_true)
counter = 0
for model_key in model_dict:
    y_score_df_dict[f"{model_key}_scores"] = roc_curve_results_dict_dict_dict[mode][model_key]["y_scores"]
    print(type(y_score_df_dict[f"{model_key}_scores"]))
    print(y_score_df_dict[f"{model_key}_scores"].numpy())
    y_score_total = y_score_total + y_score_df_dict[f"{model_key}_scores"].numpy()
    counter += 1
    
y_proba_1 = y_score_total / counter
y_proba_1

<class 'torch.Tensor'>
[0.342102   0.34332302 0.27164108 ... 0.3626734  0.34408337 0.34302092]
<class 'torch.Tensor'>
[0.21445027 0.2545019  0.22234322 ... 0.37312233 0.36723548 0.3451588 ]
<class 'torch.Tensor'>
[0.2426524  0.24578778 0.29382905 ... 0.36814204 0.36421382 0.35668382]


array([0.26640155, 0.28120424, 0.26260445, ..., 0.36797926, 0.35851089,
       0.34828784])

In [109]:
auroc = AUROC(pos_label=1)
auroc.update(torch.from_numpy(y_proba_1), torch.from_numpy(y_true))
auroc_value = auroc.compute()
auroc.reset()
print(f"auroc_value = {auroc_value:.4f}")

auroc_value = 0.9522


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [111]:
fpr, tpr, thresholds = roc_curve(y_true, y_proba_1, pos_label=1)
roc_curve_df = pd.DataFrame([fpr, tpr, thresholds]).T
roc_curve_df.columns=["fpr", "tpr", "thresholds"]
roc_curve_df

,fpr,tpr,thresholds
0,0.000000,0.000000,1.875452
1,0.000000,0.000022,0.875452
2,0.000000,0.022129,0.793947
3,0.000015,0.022129,0.793917
4,0.000015,0.032364,0.780484
...,...,...,...
16248,0.959921,0.999978,0.066508
16249,0.959921,1.000000,0.066507
16250,0.964018,1.000000,0.065718
16251,0.964047,1.000000,0.065716


In [114]:
target_fpr = 0.05
# target_fpr = 0.10
resulting_tpr = roc_curve_df.query(f"abs(fpr - {target_fpr}) < 0.0001")["tpr"].mean()
resulting_tpr

0.8513353366308355

### Learn an logistic regression

#### Get ID dict

In [124]:
mode = "val"

dataset_plot = dataset_dict[mode]
dataset_id_dict = dataset_id_dict_dict[mode]
idx_list = np.arange(len(dataset_plot))

short_identifier_list = dataset_plot.short_identifier_list
error_df_dict = dict.fromkeys(short_identifier_list)
for k in error_df_dict:
    error_df_dict[k] = []
    
for i, idx in enumerate(idx_list):
    ecg_signal, label = dataset_plot[idx]
    id_dict = Lookup_ECG(ecg_signal, dataset_id_dict)
    for k, v in id_dict.items():
        error_df_dict[k].append(v)
        
error_df = pd.DataFrame.from_dict(error_df_dict)
error_df

,patient_ID,interval_ID,block_ID,channel_ID,r_ID_abs,label,r_ID_abs_ref
0,387479,8749321,1,2,1310,1,1310
1,387479,8749321,1,2,1446,1,1446
2,387479,8749321,1,2,1583,1,1583
3,387479,8749321,1,2,1721,1,1721
4,387479,8749321,1,2,1856,1,1856
...,...,...,...,...,...,...,...
44718,462229,8748919,1,2,979966,0,979966
44719,462229,8748919,1,2,980139,0,980139
44720,462229,8748919,1,2,980312,0,980312
44721,462229,8748919,1,2,980485,0,980485


In [125]:
list(model_dict.keys())[0]

'ClocsCNN_0'

In [138]:
mode = "val"

y_score_df_dict = {}
y_true = roc_curve_results_dict_dict_dict[mode][list(model_dict.keys())[0]]["y_true"]
y_score_df_dict["y_true"] = y_true
for model_key in model_dict:
    y_score_df_dict[f"{model_key}_scores"] = roc_curve_results_dict_dict_dict[mode][model_key]["y_scores"]
y_score_df = pd.DataFrame.from_dict(y_score_df_dict)
y_score_df

,y_true,ClocsCNN_0_scores,ClocsCNN_1_scores,ClocsCNN_2_scores
0,1,0.223749,0.186808,0.227938
1,1,0.271489,0.240836,0.313976
2,1,0.241389,0.253965,0.371690
3,1,0.242686,0.201420,0.323892
4,1,0.425792,0.317882,0.285262
...,...,...,...,...
44718,0,0.263391,0.275472,0.278898
44719,0,0.274620,0.273102,0.279030
44720,0,0.279098,0.273716,0.267360
44721,0,0.288015,0.276271,0.270577


In [127]:
feature_name_list = [f"{ele}_scores" for ele in list(model_dict.keys())]
feature_name_list

['ClocsCNN_0_scores', 'ClocsCNN_1_scores', 'ClocsCNN_2_scores']

In [128]:
X_train = y_score_df[feature_name_list]
y_train = y_score_df["y_true"]
patient_ID_train = error_df["patient_ID"]
X_train

,ClocsCNN_0_scores,ClocsCNN_1_scores,ClocsCNN_2_scores
0,0.223749,0.186808,0.227938
1,0.271489,0.240836,0.313976
2,0.241389,0.253965,0.371690
3,0.242686,0.201420,0.323892
4,0.425792,0.317882,0.285262
...,...,...,...
44718,0.263391,0.275472,0.278898
44719,0.274620,0.273102,0.279030
44720,0.279098,0.273716,0.267360
44721,0.288015,0.276271,0.270577


In [130]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV


In [131]:
leave_one_group_out_cv = LeaveOneGroupOut()
tuned_parameters = [{'C': [1 / 1e-8, 1 / 0.0001, 1 / 0.001, 1 / 0.01, 1 / 0.1, 1 / 1, 1 / 10, 1 / 100, 1 / 1000, 1 / 10000]}] # sklearn uses 1 / C where C is the regularization strength

score = "accuracy"
print(f"# Tuning hyper-parameters for {score}\n")
clf = GridSearchCV(
    LogisticRegression(solver="saga", random_state=0, penalty="l1"), tuned_parameters, scoring=score, cv=leave_one_group_out_cv)

""" Run """
clf.fit(X_train, y_train, groups=patient_ID_train.values)

# Tuning hyper-parameters for accuracy



GridSearchCV(cv=LeaveOneGroupOut(),
             estimator=LogisticRegression(penalty='l1', random_state=0,
                                          solver='saga'),
             param_grid=[{'C': [100000000.0, 10000.0, 1000.0, 100.0, 10.0, 1.0,
                                0.1, 0.01, 0.001, 0.0001]}],
             scoring='accuracy')

In [137]:
print("Best parameters set found on development set:\n")
print(f"C = {1 / clf.best_params_['C']:.2f}\n")
print("Grid scores on development set (using 5-fold cross validation):\n")
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print(f"{mean:.4f} (+/-{std * 3:.4f}) for {1 / params['C']:.2f}")

print("\nDetailed classification report:\n")
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
y_true_series, y_pred = y_test, clf.predict(X_test)
y_score = clf.best_estimator_.predict_proba(X_test)
best_auroc = roc_auc_score(y_true_series, y_score[:, 1])
print(f"Best roc_auc_score = {best_auroc:.4f}\n")
print(classification_report(y_true_series, y_pred))

Best parameters set found on development set:

C = 10.00

Grid scores on development set (using 5-fold cross validation):

0.7728 (+/-0.4910) for 0.00
0.7728 (+/-0.4910) for 0.00
0.7728 (+/-0.4910) for 0.00
0.7728 (+/-0.4911) for 0.01
0.7728 (+/-0.4908) for 0.10
0.7731 (+/-0.4889) for 1.00
0.7861 (+/-0.4318) for 10.00
0.7531 (+/-0.2291) for 100.00
0.6733 (+/-0.5160) for 1000.00
0.7188 (+/-0.6241) for 10000.00

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.
Best roc_auc_score = 0.9453

              precision    recall  f1-score   support

           0       0.67      1.00      0.80     68589
           1       0.99      0.27      0.42     45823

    accuracy                           0.70    114412
   macro avg       0.83      0.63      0.61    114412
weighted avg       0.80      0.70      0.65    114412



In [133]:
y_proba = clf.best_estimator_.predict_proba(X_train)
y_proba_1 = y_proba[:, 1]
y_proba_1

array([0.00215917, 0.00571975, 0.00473447, ..., 0.00531418, 0.00606967,
       0.00486717], dtype=float32)

In [139]:
auroc = AUROC(pos_label=1)
auroc.update(torch.from_numpy(y_proba_1), torch.from_numpy(y_true))
auroc_value = auroc.compute()
auroc.reset()
print(f"auroc_value = {auroc_value:.4f}")

auroc_value = 0.9511


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


#### Test

In [140]:
mode = "test"

y_score_df_dict = {}
y_true = roc_curve_results_dict_dict_dict[mode][list(model_dict.keys())[0]]["y_true"]
y_score_df_dict["y_true"] = y_true
for model_key in model_dict:
    y_score_df_dict[f"{model_key}_scores"] = roc_curve_results_dict_dict_dict[mode][model_key]["y_scores"]
y_score_df = pd.DataFrame.from_dict(y_score_df_dict)
y_score_df

,y_true,ClocsCNN_0_scores,ClocsCNN_1_scores,ClocsCNN_2_scores
0,0,0.342102,0.214450,0.242652
1,0,0.343323,0.254502,0.245788
2,0,0.271641,0.222343,0.293829
3,0,0.319520,0.273118,0.311018
4,0,0.260570,0.191390,0.241979
...,...,...,...,...
114407,0,0.351892,0.378931,0.356382
114408,0,0.341163,0.352873,0.344683
114409,0,0.362673,0.373122,0.368142
114410,0,0.344083,0.367235,0.364214


In [142]:
X_test = y_score_df[feature_name_list]
y_test = y_score_df["y_true"]
y_proba = clf.best_estimator_.predict_proba(X_test)
y_proba_1 = y_proba[:, 1]
auroc = AUROC(pos_label=1)
auroc.update(torch.from_numpy(y_proba_1), torch.from_numpy(y_true))
auroc_value = auroc.compute()
auroc.reset()
print(f"auroc_value = {auroc_value:.4f}")

auroc_value = 0.9453


C:\ProgramData\Anaconda3\envs\apy3.10\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [143]:
fpr, tpr, thresholds = roc_curve(y_true, y_proba_1, pos_label=1)
roc_curve_df = pd.DataFrame([fpr, tpr, thresholds]).T
roc_curve_df.columns=["fpr", "tpr", "thresholds"]
roc_curve_df

,fpr,tpr,thresholds
0,0.000000,0.000000,1.996766
1,0.000000,0.000022,0.996766
2,0.000000,0.007769,0.991547
3,0.000000,0.007813,0.991547
4,0.000000,0.009624,0.991076
...,...,...,...
18855,0.996530,0.999956,0.000110
18856,0.996530,0.999978,0.000110
18857,0.999125,0.999978,0.000102
18858,0.999125,1.000000,0.000102


In [145]:
target_fpr = 0.05
# target_fpr = 0.10
resulting_tpr = roc_curve_df.query(f"abs(fpr - {target_fpr}) < 0.0001")["tpr"].mean()
resulting_tpr

0.814143552364533